# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
CLIENT:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Region:
  0: LIEU(TEXT)
  1: REGION(TEXT)
Sale:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
Wine1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wine2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


Again, we will use **`%%sql`** magic for our queries

In [4]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [5]:
%%sql 
------ MASTER 1

DROP TABLE IF EXISTS Producer;
DROP TABLE IF EXISTS Wine;
DROP TABLE IF EXISTS Production;

-- Create Producer table
CREATE TABLE Producer AS
SELECT  DISTINCT NP,  NOM,  PRENOM,  REGION
FROM MASTER1
WHERE NP IS NOT NULL;

-- Create Wine1 table
CREATE TABLE Wine1 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;

--Create Production table
CREATE TABLE Production AS
SELECT NP, NV, QTE
FROM MASTER1
WHERE QTE IS NOT NULL 
AND NP IS NOT NULL 
AND NV IS NOT NULL;

------- MASTER 2

DROP TABLE IF EXISTS Client;
DROP TABLE IF EXISTS Wine;
DROP TABLE IF EXISTS Sale;
DROP TABLE IF EXISTS Region;

-- Create Client table
CREATE TABLE CLIENT AS
SELECT  DISTINCT NB,  NOM,  PRENOM,  TYPE
FROM MASTER2
WHERE NB IS NOT NULL;

-- Create Wine2 table
CREATE TABLE Wine2 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2
WHERE NV IS NOT NULL;

--Create Sale table
CREATE TABLE Sale AS
SELECT NB, NV, DATES, LIEU, QTE
FROM MASTER2
WHERE NB IS NOT NULL
AND NV IS NOT NULL
AND DATES IS NOT NULL
AND LIEU IS NOT NULL;

--Create Region Table
CREATE TABLE Region AS
SELECT DISTINCT LIEU, REGION
FROM MASTER2
WHERE LIEU IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
(sqlite3.OperationalError) table Wine1 already exists
[SQL: -- Create Wine1 table
CREATE TABLE Wine1 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;]
(Background on this error at: http://sqlalche.me/e/e3q8)


___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [6]:
%%sql
DROP VIEW IF EXISTS bons_buveurs;
CREATE VIEW bons_buveurs AS
SELECT * FROM Client
WHERE TYPE IN ('moyen', 'gros');

 * sqlite:///wine.db
Done.
Done.


[]

In [7]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,gros
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,gros
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [8]:
%%sql
DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS
SELECT CLient.NB, NOM, PRENOM, TYPE 
FROM Client LEFT OUTER JOIN Sale ON (Client.NB = Sale.NB)
WHERE NV is null;

 * sqlite:///wine.db
Done.
Done.


[]

In [9]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [10]:
%%sql
DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS
SELECT DISTINCT CLient.NB, NOM, PRENOM, TYPE 
FROM Client JOIN Sale ON (Client.NB = Sale.NB);

 * sqlite:///wine.db
Done.
Done.


[]

In [11]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,gros
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,gros
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,gros
10,Andersen,Yann,gros


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [12]:
%%sql
DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE 
FROM Wine2 
JOIN Sale ON (Sale.NV = Wine2.NV)
JOIN Client ON (Client.NB = Sale.NB)
WHERE DATES LIKE "1983%"
GROUP BY LIEU, CRU
ORDER BY LIEU, CRU;

 * sqlite:///wine.db
Done.
Done.


[]

In [13]:
# Test
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

Answer: Yes if you mean "QTE_BUE" we can we just add the QTE as the first element of the ORDER BY option to be sure it is firstly order by QTE as in the following request.
If you mean "QTE" it would not work because the attributes QTE does not have a meaning once we make our GROUP BY on LIEU and CRU.

In [14]:
%%sql
SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE 
FROM Wine2 
JOIN Sale ON (Sale.NV = Wine2.NV)
JOIN Client ON (Client.NB = Sale.NB)
WHERE DATES LIKE "1983%"
GROUP BY LIEU, CRU
ORDER BY QTE_BUE, LIEU, CRU;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
PARIS,Coteaux du Tricastin,1
RENNES,Mercurey,1
LYON,Julienas,2
CAEN,Seyssel,3
PARIS,Pouilly Vinzelles,3
PARIS,Beaujolais Primeur,4
LILLE,Pommard,5
LYON,Beaujolais Villages,10
ROCQUENCOURT,Saint Amour,80
ROCQUENCOURT,Beaujolais Villages,260


In [15]:
%%sql
SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE 
FROM Wine2 
JOIN Sale ON (Sale.NV = Wine2.NV)
JOIN Client ON (Client.NB = Sale.NB)
WHERE DATES LIKE "1983%"
GROUP BY LIEU, CRU
ORDER BY QTE, LIEU, CRU;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
PARIS,Coteaux du Tricastin,1
RENNES,Mercurey,1
LYON,Julienas,2
CAEN,Seyssel,3
PARIS,Pouilly Vinzelles,3
PARIS,Beaujolais Primeur,4
LILLE,Pommard,5
LYON,Beaujolais Villages,10
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


In [16]:
%%sql
SELECT LIEU, CRU,QTE
FROM Wine2 
JOIN Sale ON (Sale.NV = Wine2.NV)
JOIN Client ON (Client.NB = Sale.NB)
WHERE DATES LIKE "1983%"
ORDER BY QTE, LIEU, CRU;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE
PARIS,Coteaux du Tricastin,1
RENNES,Mercurey,1
LYON,Julienas,2
CAEN,Seyssel,3
PARIS,Pouilly Vinzelles,3
PARIS,Beaujolais Primeur,4
LILLE,Pommard,5
LYON,Beaujolais Villages,10
ROCQUENCOURT,Beaujolais Villages,10
ROCQUENCOURT,Saint Amour,80


___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [17]:
%%sql
DROP TABLE IF EXISTS RBB;
CREATE TABLE RBB AS 
SELECT * FROM bons_buveurs
ORDER BY NB;

 * sqlite:///wine.db
Done.
Done.


[]

In [18]:
# Test
%sql SELECT * FROM RBB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,gros
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,gros
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen


#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [19]:
%%sql
SELECT Client.NB, NOM, PRENOM, TYPE, SUM(QTE) AS TOTAL
FROM Client
JOIN Sale ON Client.NB = Sale.NB
WHERE TYPE <> 'gros'
GROUP BY Client.NB, NOM, PRENOM, TYPE
HAVING SUM(QTE) > 100;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,TOTAL


Update instances

In [20]:
%%sql
UPDATE Client
SET TYPE = 'gros'
WHERE NB IN (
SELECT Client.NB
FROM Client
JOIN Sale ON Client.NB = Sale.NB
WHERE TYPE <> 'gros'
GROUP BY Client.NB, NOM, PRENOM, TYPE
HAVING SUM(QTE) > 100
);

 * sqlite:///wine.db
0 rows affected.


[]

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

In [21]:
%%sql
SELECT RBB.NB, NOM, PRENOM, TYPE, SUM(QTE) AS TOTAL
FROM RBB
JOIN Sale ON RBB.NB = Sale.NB
WHERE TYPE <> 'gros'
GROUP BY RBB.NB, NOM, PRENOM, TYPE
HAVING SUM(QTE) > 100;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,TOTAL


In [22]:
%%sql
SELECT bons_buveurs.NB, NOM, PRENOM, TYPE, SUM(QTE) AS TOTAL
FROM bons_buveurs
JOIN Sale ON bons_buveurs.NB = Sale.NB
WHERE TYPE <> 'gros'
GROUP BY bons_buveurs.NB, NOM, PRENOM, TYPE
HAVING SUM(QTE) > 100;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,TOTAL


**bons_buveurs** and **RBB** only have Client of type *moyen* and *gros*. So after the update of our table **Client** only the line of NB=2 which is of type *moyen* will have an impact on those table.

We can observe that after the update only the VIEW **bons_buveurs** is affected by the change. It is based on the creation of views which are a copy of a table that is not store in memory, so the views change with the table it was bult on.

On the other hand the table **RBB** is stored in memory and doesn't keep any link with the view **bons_buveurs** it was built on, that is why the value of the client NB = 2 is still visible after the update.

#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [23]:
%%sql
DROP TABLE IF EXISTS RBA;
CREATE TABLE RBA AS 
SELECT * FROM buveurs_asec
ORDER BY NB;

 * sqlite:///wine.db
Done.
Done.


[]

In [24]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [25]:
%%sql
INSERT INTO RBA (NB, NOM, PRENOM, TYPE)  
VALUES (101, 'richard', 'vincent', 'moyen');

 * sqlite:///wine.db
1 rows affected.


[]

In [51]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

In [27]:
%sql SELECT * FROM RBA WHERE NB=101;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
101,richard,vincent,moyen


In [28]:
%sql SELECT * FROM buveurs_asec WHERE NB=101;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE


Here we insert a new value the table **RBA** built on our view **buveurs_asec**, this modification will not have a impact on the value of our view because the view are only link to the table they were created from, not the tables that where build on them. 

Which means if we want this new line in **buveurs_asec** we will have to insert a new line in the tables **Client** and **Sale**.

___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

<img src="socialNetworkSchema.png" width="800" height="400">

I used 4 tables:

 + **user**: It's the main table the primaty key is *nickname* with the hypothesis that two users can't have the same nickname. <br>
     We also put information about the user, like *name*, *email*, and the *date of birth*. I decided to put the adress in the same table since it is generated by the user, it may contain error or false adress.
 + **friend**: The table containing couple of nickname each user as a line per friend he has. We can create one row per friendship or two with each possible combinaison (we keep this option)
 + **post**: Keeping all information of a post. Since a user can create two identical posts (we consider the time by minute), we create a id as the primary key. We put all information about the post except the type.
 + **type_post**: This class is use to link reply and original post. We have the *id_post* and *type*. If the *type* is 'reply' then it will have a *id_original_post* to keep the information of which post it was posted on. If the *type* is 'original', we have to choice, we can either put a null value in *id_orignal_post* or put the again it's own id *id_post*. The null value seems a better option.

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.

A first simple view to create without the issue of having too much friend would be:
```sql
CREATE VIEW post_to_display AS
SELECT post.nickname_author, post.text, post.date, post.attachement
FROM post
    JOIN type_post ON (post.id = type_post.id_post)
    JOIN friend ON (user.nickname = friend.nickname_1)
WHERE friend.nickname_2 == 'nickname_of_user_loged_in' -- post of the friend of the user
    AND type_post.type == 'original' -- only post and not response
ORDER BY post.date DESC;
```
Which means getting all **original** post from the friend of the user loged in (this value will be a variable that change for each user that we symbolize here by 'nickname_of_user_loged_in'), and order them by their **date** of post. We don't display reply on the view, base on the idea that it could be display if the original post is clicked on, so if the user is interested in it.
 
If we want to limit the number of post display by 20 and let the user click a button to add 20 more and so on. We need to add a way to display interesting post first. To do so we can add a table **view_post** that keep a link between users and the posts seen. In this way we can promote the post he didn't see first. Of course those post must be recent (up to 2-3 days for example) otherwise post that the user already seen but want to see again will be lost.

If we add a new field *click_post* on **view_post** that keep the information about each post a user has click, we can display the post that the friends of the user have clicked on but the current user didn't see yet. Because we can assume that if his friends has click on a post, that mean they were interested in it and so it might interest the current user.

To order our post we will turn a sql script that will set the value of the post to see first. This value will be by default high. We name it here *interset* and we put it in a table .

Request to run before (Order by select doesn't work but I don't know any way to do it, we might be able to do so by creating multiple views and joining them while keeping an attributes that give us the order)


```sql
CREATE VIEW post_to_display AS
SELECT post.nickname_author, post.text, post.date, post.attachement
FROM post
    JOIN type_post ON (post.id = type_post.id_post)
    JOIN friend ON (user.nickname = friend.nickname_1)
WHERE friend.nickname_2 == 'nickname_of_user_loged_in' -- post of the friend of the user
    AND type_post.type == 'original' -- only post and not response
ORDER BY
    ( --  sort unseen post of the last three days first take only 10 to see old post already seen
    SELECT view_post.id_post 
    FROM view_post
    WHERE view_post.nickname == 'nickname_of_user_loged_in'
        AND view_post.view == 1
        AND (post.date - today) < three_days
    LIMIT 10),
   ( --sort post that friends have click on during the last 24 hours and order them by date take only 5 of them to have new posts
    SELECT view_post.id_post 
    FROM view_post 
        JOIN friend ON (view_post.nickname = friend.nickname_2)
        JOIN post ON (post.id = view_post.id_post)
    WHERE friend.nickname_1 == 'nickname_of_user_loged_in' 
        AND click_post == 1
        AND post.date - today) < yesterday
    ORDER BY post.date DESC
    LIMIT 5
    ),
    post.date DESC
LIMIT 20; 
```